In [1]:
# Import public packages and functions
import os
import pandas as pd
import numpy as np
import sys
import json
from pathlib import Path
from scipy import signal

import seaborn as sns
import matplotlib.pyplot as plt
import scikit_posthocs as sp

import warnings
warnings.filterwarnings("ignore")

# inserting the lib folder to the compiler
sys.path.insert(0, './lib')
sys.path.insert(0, './utils/')

import utils_io, utils_phase_analysis

from lib_data import DATA_IO

In [2]:
PATH_CURR = os.path.abspath(os.curdir)    # current code
PATH      = (str(Path(PATH_CURR).parent)) # data repository: upper directory where datasets situated
fs        = 2048

## 1. Load Events

In [5]:
# load ECoG events
EVENTS_ECOG            = utils_io.load_ECoG_events(event_category="tapping", fs=2048)
EVENTS_ECOG_LID        = EVENTS_ECOG['controlateral']['LID']
EVENTS_ECOG_noLID      = EVENTS_ECOG['controlateral']['noLID']

EVENTS_ECOG_LID["hemisphere"]        = EVENTS_ECOG_LID["ECoG_hemisphere"] 
EVENTS_ECOG_LID["channel"]           = EVENTS_ECOG_LID["ECoG_channel"] 
EVENTS_ECOG_noLID["hemisphere"]      = EVENTS_ECOG_noLID["ECoG_hemisphere"] 
EVENTS_ECOG_noLID["channel"]         = EVENTS_ECOG_noLID["ECoG_channel"] 

relevant_features  = ['patient', 'event_no', 'event_category', 'event_laterality', 'event_start_time', 'duration', 'hemisphere', 
                      'channel','pre_event_recording', 'event_recording', 'post_event_recording', "event_recording_onset_alingned"]
noLID_noDOPA_ECOG  = EVENTS_ECOG_noLID[EVENTS_ECOG_noLID.event_start_time <= 30][relevant_features]
noLID_DOPA_ECOG    = EVENTS_ECOG_noLID[EVENTS_ECOG_noLID.event_start_time >  30][relevant_features]
LID_ECOG           = EVENTS_ECOG_LID[relevant_features]

# 1.1 Measure Baseline ERPAC

In [3]:
def split_baseline_into_segments(recording, fs=2048, segment_duration=2):
    recording           = np.array(recording)
    recording           = recording[~np.isnan(recording)]       # remove the np.nan segments
    samples_per_segment = int(fs * segment_duration)
    total_segments      = len(recording) // samples_per_segment # trim the signal so it divides evenly
    trimmed_signal      = recording[:total_segments * samples_per_segment]
    segments            = trimmed_signal.reshape(total_segments, samples_per_segment)
    return segments

def mean_erpac(arr):
    arr = np.squeeze(arr)           # (6, 4096)
    arr = np.nanmean(arr, axis=1)   # (6,)
    return arr

In [7]:
ECOG_baseline         = utils_io.load_baseline_recordings("ECoG")
df_baseline_avg_erpac = []

for patient in ECOG_baseline.keys():
    for hemisphere in ECOG_baseline[patient].keys():
        for channel in ECOG_baseline[patient][hemisphere].keys():
            baseline_recording      = ECOG_baseline[patient][hemisphere][channel]
            baseline_segments       = split_baseline_into_segments(baseline_recording)
            baseline_erpac          = utils_phase_analysis.channel_wise_ERPAC(baseline_segments, patient, hemisphere, channel, "baseline", fs)
            baseline_erpac['erpac'] = baseline_erpac['erpac'].apply(mean_erpac)
            df_baseline_avg_erpac.append(baseline_erpac)
    print(f"patient {patient} is completed!") 
    
df_baseline_avg_erpac = pd.concat(df_baseline_avg_erpac, ignore_index=True)
df_baseline_avg_erpac.to_pickle(DATA_IO.path_events + f"erpac/ECOG/baseline.pkl")

Patient 008: does not have right hemisphere baseline recordings
Patient 009: does not have right hemisphere baseline recordings
Patient 010: does not have left hemisphere baseline recordings
Patient 012: does not have left hemisphere baseline recordings
Patient 013: does not have left hemisphere baseline recordings
Patient 014: does not have left hemisphere baseline recordings
Patient 016: does not have left hemisphere baseline recordings
Patient 017: does not have left hemisphere baseline recordings


Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=6)


Patient 020: does not have left hemisphere baseline recordings
Patient 021: does not have left hemisphere baseline recordings
Patient 022: does not have left hemisphere baseline recordings
Patient 023: does not have left hemisphere baseline recordings
Patient 101: does not have right hemisphere baseline recordings
Patient 101: does not have left hemisphere baseline recordings
Patient 102: does not have right hemisphere baseline recordings
Patient 102: does not have left hemisphere baseline recordings
Patient 103: does not have right hemisphere baseline recordings
Patient 103: does not have left hemisphere baseline recordings
Patient 105: does not have right hemisphere baseline recordings
Patient 105: does not have left hemisphere baseline recordings
Patient 107: does not have right hemisphere baseline recordings
Patient 107: does not have left hemisphere baseline recordings
Patient 108: does not have right hemisphere baseline recordings
Patient 108: does not have left hemisphere baseli

    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

patient 008 is completed!


...

patient 021 is completed!


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

patient 022 is completed!


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

patient 023 is completed!


# 2. ERPAC Measurement

In [9]:
dataset        = noLID_noDOPA_ECOG
severity       = "noLID_noDOPA"
recording_type = "ECOG"

for patient in dataset.patient.unique():
    dataset_patient = dataset[dataset.patient==patient]
    patient_ERPAC   = utils_phase_analysis.get_ERPAC_for_dyskinesia_severity(dataset=dataset_patient, recording_type=recording_type, severity=severity, fs=fs)
    patient_ERPAC.to_pickle(DATA_IO.path_events + f"erpac/{recording_type}/{severity}/{patient}.pkl")

Event Related PAC object defined


------------------------------------------------------------------------
------------------------------------------------------------------------
             EVENT-RELATED PHASE AMPLITUDE COUPLING ANALYSIS            
------------------------------------------------------------------------
------------------------------------------------------------------------
recording : ECOG
severity  : noLID_noDOPA
>>> patient : 009 - hemisphere : left - channel : 02-01


    Extract phases (n_pha=1) and amplitudes (n_amps=6)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phase

>>> >>> onset aligned recordings: ERPAC estimation completed for the channel!


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

...

>>> >>> offset aligned recordings: ERPAC estimation completed for the channel!
>>> patient : 023 - hemisphere : right - channel : 06-05


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

>>> >>> onset aligned recordings: ERPAC estimation completed for the channel!


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

>>> >>> offset aligned recordings: ERPAC estimation completed for the channel!


In [10]:
dataset        = noLID_DOPA_ECOG
severity       = "noLID_DOPA"
recording_type = "ECOG"

for patient in dataset.patient.unique():
    dataset_patient = dataset[dataset.patient==patient]
    patient_ERPAC   = utils_phase_analysis.get_ERPAC_for_dyskinesia_severity(dataset=dataset_patient, recording_type=recording_type, severity=severity, fs=fs)
    patient_ERPAC.to_pickle(DATA_IO.path_events + f"erpac/{recording_type}/{severity}/{patient}.pkl")

Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=6)


------------------------------------------------------------------------
------------------------------------------------------------------------
             EVENT-RELATED PHASE AMPLITUDE COUPLING ANALYSIS            
------------------------------------------------------------------------
------------------------------------------------------------------------
recording : ECOG
severity  : noLID_DOPA
>>> patient : 010 - hemisphere : right - channel : 02-01


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

>>> >>> onset aligned recordings: ERPAC estimation completed for the channel!


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

...

>>> >>> offset aligned recordings: ERPAC estimation completed for the channel!
>>> patient : 023 - hemisphere : right - channel : 06-05


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

>>> >>> onset aligned recordings: ERPAC estimation completed for the channel!


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

>>> >>> offset aligned recordings: ERPAC estimation completed for the channel!


In [11]:
dataset        = LID_ECOG
severity       = "LID"
recording_type = "ECOG"

for patient in dataset.patient.unique():
    dataset_patient = dataset[dataset.patient==patient]
    patient_ERPAC   = utils_phase_analysis.get_ERPAC_for_dyskinesia_severity(dataset=dataset_patient, recording_type=recording_type, severity=severity, fs=fs)
    patient_ERPAC.to_pickle(DATA_IO.path_events + f"erpac/{recording_type}/{severity}/{patient}.pkl")

Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=6)


------------------------------------------------------------------------
------------------------------------------------------------------------
             EVENT-RELATED PHASE AMPLITUDE COUPLING ANALYSIS            
------------------------------------------------------------------------
------------------------------------------------------------------------
recording : ECOG
severity  : LID
>>> patient : 008 - hemisphere : left - channel : 02-01


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

>>> >>> onset aligned recordings: ERPAC estimation completed for the channel!


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

...

>>> >>> offset aligned recordings: ERPAC estimation completed for the channel!
>>> patient : 023 - hemisphere : right - channel : 06-05


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

>>> >>> onset aligned recordings: ERPAC estimation completed for the channel!


    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=14)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=28)
    Compute Gaussian-Copula ERPAC
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=58)
    Compute Gaus

>>> >>> offset aligned recordings: ERPAC estimation completed for the channel!


# 3. ERPAC Load

In [ ]:
rp_obj                      = EventRelatedPac(f_pha=band_descriptions[phase_band], f_amp=band_descriptions[amplitude_band])
erpac                       = rp_obj.filterfit(fs, channel_data, method='gc', smooth=100)

In [ ]:
plt.figure(figsize=(8, 6))
rp_obj.pacplot(erpac.squeeze(), t, rp_obj.yvec, xlabel='Time',
               ylabel='Amplitude frequency (Hz)',
               title='Event-Related PAC occurring for alpha phase',
               fz_labels=15, fz_title=18)
#add_motor_condition(135, color='white')
plt.show()

In [ ]:
np.shape(erpac)